In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import copy

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices
from src.gcforest.gcforest import GCForest
from src.gcforest.utils.config_utils import load_json

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [20]:
def load_understand_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
#     print(cols_list)
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def load_commit_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    commit_guru_df = pd.read_csv(commit_guru_path)
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix','fixes',],axis=1)

    df = commit_guru_df
    df.rename(columns={"contains_bug": "Bugs"},inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    y.fillna(False,inplace=True)
    df['Bugs'] = y
    df.dropna(inplace=True)
    cols = df.columns.tolist()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
#     df,cols = apply_cfs(df)
#     y = df.Bugs
#     X = df.drop('Bugs',axis = 1)
    return df

def normalize(X):
    commit_hash = X.commit_hash
    X = X.drop('commit_hash',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    X['commit_hash'] = commit_hash
    return X

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 100
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 2
    ca_config["estimators"] = []
#     ca_config["estimators"].append(
#             {"n_folds": 5, "type": "XGBClassifier", "n_estimators": 10, "max_depth": 5,
#              "objective": "multi:softprob", "silent": True, "nthread": -1, "learning_rate": 0.1} )
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "n_estimators": 100, "max_depth": None, "n_jobs": -1})
#     ca_config["estimators"].append({"n_folds": 5, "type": "ExtraTreesClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
#     ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config

In [13]:
def norm(x,df):
    lo = df.min()
    hi = df.max()
    return (x - lo) / (hi - lo +0.00000001)

def dominate(_df,t,row_project_name,goals):
    wins = 0
    for i in range(_df.shape[0]):
        project_name = _df.index[i]
        row = _df.iloc[i].tolist()
        if project_name != row_project_name:
            if dominationCompare(row, t,goals,_df):
                wins += 1
    return wins

def dominationCompare(other_row, t,goals,df):
    n = len(goals)
    weight = {'recall':1,'precision':1,'pf':-2.5}
    sum1, sum2 = 0,0
    for i in range(len(goals)):
        _df = df[goals[i]]
        w = weight[goals[i]]
        x = t[i]
        y = other_row[i]
        x = norm(x,_df)
        y = norm(y,_df)
        sum1 = sum1 - math.e**(w * (x-y)/n)
        sum2 = sum2 - math.e**(w * (y-x)/n)
    return sum1/n < sum2/n

In [14]:
def run_self(project):
    commit_X,commit_y = load_commit_data(project)
    understand_X,understand_y = load_understand_data(project)
    print(commit_X)
    
    
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
#     df_smote = pd.concat([X_train,y_train],axis = 1)
#     df_smote = apply_smote(df_smote)
#     y_train = df_smote.Bugs
#     X_train = df_smote.drop('Bugs',axis = 1)
# #     clf = LogisticRegression(penalty='l1',solver='liblinear')
# #     clf = Earth()
#     clf = RandomForestClassifier()
#     clf.fit(X_train,y_train)
# #     print(clf.coef_)
#     predicted = clf.predict(X_test)
# #     print(predicted)
#     abcd = metrices.measures(y_test,predicted)
#     pf = abcd.get_pf()
#     recall = abcd.calculate_recall()
#     precision = abcd.calculate_precision()
#     f1 = abcd.calculate_f1_score()
#     g_score = abcd.get_g_score()
#     try:
#         auc = roc_auc_score(y_test, predicted)
#     except:
#         auc = 0
#     print(classification_report(y_test, predicted))
#     return recall,precision,pf,f1,g_score,auc

In [15]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [21]:

for project in projects:
    if project != 'parquet-mr':
        continue
    
    # Process Data
    commit_df = load_commit_data(project)
    commit_y = commit_df.Bugs
    commit_X = commit_df.drop('Bugs',axis = 1)
    commit_X = normalize(commit_X)
    commit_X_train, commit_X_test, commit_y_train, commit_y_test = train_test_split(commit_X, 
                                                                                    commit_y, 
                                                                                    test_size=0.40, 
                                                                                    random_state=18)
    
    # Product Data
    understand_df = load_understand_data(project)
    understand_y = understand_df.Bugs
    understand_X = understand_df.drop('Bugs',axis = 1)
    understand_X = normalize(understand_X)
    understand_df['Bugs'] = understand_y
    understand_df_train = understand_df[understand_df['commit_hash'].isin(commit_X_train.commit_hash)]
    understand_df_test = understand_df[understand_df['commit_hash'].isin(commit_X_test.commit_hash)]
    understand_y_train = understand_df_train.Bugs
    understand_X_train = understand_df_train.drop('Bugs',axis = 1)
    understand_y_test = understand_df_test.Bugs
    understand_X_test = understand_df_test.drop('Bugs',axis = 1)
    
    
    # Combined Data
    combined_df = understand_df.merge(commit_X,on='commit_hash')
    cols = combined_df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    combined_df = combined_df[cols]
    combined_df_train = combined_df[combined_df['commit_hash'].isin(commit_X_train.commit_hash)]
    combined_df_test = combined_df[combined_df['commit_hash'].isin(commit_X_test.commit_hash)]
    combined_y_train = combined_df_train.Bugs
    combined_X_train = combined_df_train.drop('Bugs',axis = 1)
    combined_y_test = combined_df_test.Bugs
    combined_X_test = combined_df_test.drop('Bugs',axis = 1)
    
    
    # Feature Adding to combined data
    commit_df_new = pd.concat([commit_X_train,commit_y_train],axis = 1)
    commit_df_new.rename(columns={"Bugs": "contains_bug"},inplace=True)
    featured_combined_df = understand_df_train.merge(commit_df_new,on='commit_hash')
    cols = featured_combined_df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    featured_combined_df = featured_combined_df[cols]
    featured_combined_y_train = featured_combined_df.Bugs
    featured_combined_X_train = featured_combined_df.drop('Bugs',axis = 1)
    
    
    
    # Test Data with commit_hash
    commit_X_test_w = copy.deepcopy(commit_X_test)
    
    # Removing commit hash
    commit_X_train = commit_X_train.drop('commit_hash',axis = 1)
    commit_X_test = commit_X_test.drop('commit_hash',axis = 1)
    understand_X_train = understand_X_train.drop('commit_hash',axis = 1)
    understand_X_test = understand_X_test.drop('commit_hash',axis = 1)
    combined_X_train = combined_X_train.drop('commit_hash',axis = 1)
    combined_X_test = combined_X_test.drop('commit_hash',axis = 1)
    featured_combined_X_train = featured_combined_X_train.drop('commit_hash',axis = 1)
    
    
    # Smote Process Data
    commit_df_smote = pd.concat([commit_X_train,commit_y_train],axis = 1)
    commit_df_smote = apply_smote(commit_df_smote)
    commit_y_train = commit_df_smote.Bugs
    commit_X_train = commit_df_smote.drop('Bugs',axis = 1)
    
    # Smote Product Data
    understand_df_smote = pd.concat([understand_X_train,understand_y_train],axis = 1)
    understand_df_smote = apply_smote(understand_df_smote)
    understand_y_train = understand_df_smote.Bugs
    understand_X_train = understand_df_smote.drop('Bugs',axis = 1)
    
    # Smote Combined Data
    combined_df_smote = pd.concat([combined_X_train,combined_y_train],axis = 1)
    combined_df_smote = apply_smote(combined_df_smote)
    combined_y_train = combined_df_smote.Bugs
    combined_X_train = combined_df_smote.drop('Bugs',axis = 1)
    
    # Smote Feature Combined Data
    feature_combined_df_smote = pd.concat([featured_combined_X_train,featured_combined_y_train],axis = 1)
    feature_combined_df_smote = apply_smote(feature_combined_df_smote)
    featured_combined_y_train = feature_combined_df_smote.Bugs
    featured_combined_X_train = feature_combined_df_smote.drop('Bugs',axis = 1)
    
#     # Normalize Train Data
#     commit_X_train = normalize(commit_X_train)
#     understand_X_train = normalize(understand_X_train)
#     combined_X_train = normalize(combined_X_train)
#     featured_combined_X_train = normalize(featured_combined_X_train)
    
#     # Normalize Test Data 
#     commit_X_test = normalize(commit_X_test)
#     understand_X_test = normalize(understand_X_test)
#     combined_X_test = normalize(combined_X_test)

    config = get_toy_config()
    gc = GCForest(config)
    X_train_enc = gc.fit_transform(commit_X_train.values, commit_y_train.values)
    y_pred = gc.predict(commit_X_test.values)
    print(classification_report(commit_y_test, y_pred))
    
    
    
    
    # Process Classifier
    clf_process = RandomForestClassifier()
    clf_process.fit(commit_X_train,commit_y_train)
    
    # Product Classifier
    clf_product = RandomForestClassifier()
    clf_product.fit(understand_X_train,understand_y_train)
    
    # Combined Classifier
    clf_combined = RandomForestClassifier()
    clf_combined.fit(combined_X_train,combined_y_train)
    
    # Added feature Classifier
    clf_feature_combined = RandomForestClassifier()
    clf_feature_combined.fit(featured_combined_X_train,featured_combined_y_train)
    
    # Process Prediction
    process_predicted = clf_process.predict(commit_X_test)
    print(classification_report(commit_y_test, process_predicted))
    
    # Product Prediction
    product_predicted = clf_product.predict(understand_X_test)
    print(classification_report(understand_y_test, product_predicted))
    
    # Combined Prediction
    combined_predicted = clf_combined.predict(combined_X_test)
    print(classification_report(combined_y_test, combined_predicted))
    
    # 2 stage classifier
    commit_test_df_w = copy.deepcopy(commit_X_test_w)
    commit_test_df_w['Bugs'] = process_predicted
    
    # true returned from process prediction
    commit_test_df_true = commit_test_df_w[commit_test_df_w['Bugs'] == True]
    sub_true_combined_df_test = combined_df_test[combined_df_test['commit_hash'].isin(commit_test_df_true.commit_hash)]
    sub_true_combined_y_test = sub_true_combined_df_test.Bugs
    sub_true_combined_X_test = sub_true_combined_df_test.drop('Bugs',axis = 1)
    sub_true_combined_X_test = sub_true_combined_X_test.drop('commit_hash',axis = 1)
    sub_true_combined_predicted = clf_combined.predict(sub_true_combined_X_test)
    
    # false returned from process prediction
    commit_test_df_false = commit_test_df_w[commit_test_df_w['Bugs'] == False]
    sub_false_combined_df_test = combined_df_test[combined_df_test['commit_hash'].isin(commit_test_df_false.commit_hash)]
    sub_false_combined_y_test = sub_false_combined_df_test.Bugs
    sub_false_combined_X_test = sub_false_combined_df_test.drop('Bugs',axis = 1)
    sub_false_combined_X_test = sub_false_combined_X_test.drop('commit_hash',axis = 1)
    sub_flase_combined_predicted = [0]*sub_false_combined_X_test.shape[0]
    
    final_staged_y_test = list(sub_true_combined_y_test) + list(sub_false_combined_y_test)
    final_staged_combined_predicted = list(sub_true_combined_predicted) + list(sub_flase_combined_predicted)
    
    print(classification_report(sub_true_combined_y_test, sub_true_combined_predicted))
    
    print(classification_report(final_staged_y_test, final_staged_combined_predicted))
    
    # Added feature classifier
    commit_X_test_w['contains_bug'] = process_predicted
    featured_combined_df_test = understand_df_test.merge(commit_X_test_w,on='commit_hash')
    featured_combined_df_test = featured_combined_df_test.drop('commit_hash',axis = 1)
    featured_combined_y_test = featured_combined_df_test.Bugs
    featured_combined_X_test = featured_combined_df_test.drop('Bugs',axis = 1)
#     combined_X_test = normalize(combined_X_test)
    featured_combined_predicted = clf_feature_combined.predict(featured_combined_X_test)
    print(classification_report(featured_combined_y_test, featured_combined_predicted))
    
    
    
    
    
    

[ 2020-04-02 12:29:31,555][cascade_classifier.fit_transform] X_groups_train.shape=[(1728, 13)],y_train.shape=(1728,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2020-04-02 12:29:31,557][cascade_classifier.fit_transform] group_dims=[13]
[ 2020-04-02 12:29:31,558][cascade_classifier.fit_transform] group_starts=[0]
[ 2020-04-02 12:29:31,559][cascade_classifier.fit_transform] group_ends=[13]
[ 2020-04-02 12:29:31,559][cascade_classifier.fit_transform] X_train.shape=(1728, 13),X_test.shape=(0, 13)
[ 2020-04-02 12:29:31,560][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1728, 13), X_cur_test.shape=(0, 13)
[ 2020-04-02 12:29:31,804][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_0.predict)=90.75%
[ 2020-04-02 12:29:32,052][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_1.predict)=93.93%
[ 2020-04-02 12:29:32,304][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.trai

              precision    recall  f1-score   support

       False       0.89      0.94      0.92       563
        True       0.71      0.56      0.62       142

    accuracy                           0.87       705
   macro avg       0.80      0.75      0.77       705
weighted avg       0.86      0.87      0.86       705

              precision    recall  f1-score   support

       False       0.91      0.92      0.91       563
        True       0.66      0.63      0.64       142

    accuracy                           0.86       705
   macro avg       0.78      0.77      0.78       705
weighted avg       0.86      0.86      0.86       705

              precision    recall  f1-score   support

           0       0.68      0.74      0.71       939
           1       0.67      0.59      0.63       829

    accuracy                           0.67      1768
   macro avg       0.67      0.67      0.67      1768
weighted avg       0.67      0.67      0.67      1768

              preci

In [ ]:
precision    recall  f1-score   support

       False       0.91      0.94      0.92       563
        True       0.72      0.63      0.67       142

    accuracy                           0.88       705
   macro avg       0.81      0.78      0.80       705
weighted avg       0.87      0.88      0.87       705

              precision    recall  f1-score   support

           0       0.68      0.74      0.71       939
           1       0.68      0.61      0.64       829

    accuracy                           0.68      1768
   macro avg       0.68      0.68      0.68      1768
weighted avg       0.68      0.68      0.68      1768

              precision    recall  f1-score   support

           0       0.86      0.64      0.74       939
           1       0.68      0.89      0.77       829

    accuracy                           0.76      1768
   macro avg       0.77      0.76      0.75      1768
weighted avg       0.78      0.76      0.75      1768

              precision    recall  f1-score   support

           0       0.85      0.58      0.69       641
           1       0.71      0.91      0.80       740

    accuracy                           0.76      1381
   macro avg       0.78      0.74      0.74      1381
weighted avg       0.78      0.76      0.75      1381

              precision    recall  f1-score   support

           0       0.81      0.71      0.76       939
           1       0.71      0.81      0.76       829

    accuracy                           0.76      1768
   macro avg       0.76      0.76      0.76      1768
weighted avg       0.77      0.76      0.76      1768

              precision    recall  f1-score   support

           0       0.83      0.71      0.76       939
           1       0.72      0.84      0.77       829

    accuracy                           0.77      1768
   macro avg       0.77      0.77      0.77      1768
weighted avg       0.78      0.77      0.77      1768

In [ ]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'org.alloytools.alloy':
#             continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
with open('data/self_1000.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
combined_X_train.values

In [ ]:
_precision = list(precision_list.values())
_recall = list(recall_list.values())
_pf = list(pf_list.values())
_f1 = list(f1_list.values())
_g = list(g_list.values())
_auc = list(auc_list.values())
print(np.median(_precision),np.median(_recall),np.median(_pf),np.median(_f1),np.median(_g),np.median(_auc))
fig = plt.figure(num=None, figsize = (20,4), facecolor='w', edgecolor='k')
ax = fig.add_subplot(161)
ax.boxplot(_precision)
ax.set_title('Precision',size = 15)
ax = fig.add_subplot(162)
ax.boxplot(_recall)
ax.set_title('Recall',size = 15)
ax = fig.add_subplot(163)
ax.boxplot(_pf)
ax.set_title('pf',size = 15)
ax = fig.add_subplot(164)
ax.boxplot(_f1)
ax.set_title('f1',size = 15)
ax = fig.add_subplot(165)
ax.boxplot(_g)
ax.set_title('g',size = 15)
ax = fig.add_subplot(166)
ax.boxplot(_auc)
ax.set_title('auc',size = 15)
# fig.savefig('without_process.png')

In [ ]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = LogisticRegression()
    clf.fit(X_train,y_train)
    return clf

In [ ]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
for s_project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + s_project + "  +++++++++++++++++")
        clf = run_self(s_project)
        if s_project not in precision_list.keys():
            precision_list[s_project] = {}
            recall_list[s_project] = {}
            pf_list[s_project] = {}
            f1_list[s_project] = {}
            g_list[s_project] = {}
            auc_list[s_project] = {}    
        for d_project in projects:
            try:
                X,y = load_data(d_project)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
                predicted = clf.predict(X_test)
                abcd = metrices.measures(y_test,predicted)
                pf = abcd.get_pf()
                recall = abcd.calculate_recall()
                precision = abcd.calculate_precision()
                f1 = abcd.calculate_f1_score()
                g_score = abcd.get_g_score()
                auc = roc_auc_score(y_test, predicted)
                precision_list[s_project][d_project] = precision
                recall_list[s_project][d_project] = recall
                pf_list[s_project][d_project] = pf
                f1_list[s_project][d_project] = f1
                g_list[s_project][d_project] = g_score
                auc_list[s_project][d_project] = auc
            except Exception as e:
                print('d_project',e)
                continue
    except Exception as e:
        print('s_project',e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
with open('data/conv_bellwether_100.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df_self = pd.read_pickle('data/self_100.pkl')
df_bellwether = pd.read_pickle('data/conv_bellwether_100.pkl')
metrices = df_self.keys()
median = 0
median_results = {}
for s_project in projects:
    try:
        if s_project not in median_results.keys():
            median_results[s_project] = {}
        for metric in metrices:
            median_results[s_project][metric] = np.median(list(df_bellwether[metric][s_project].values()))
    except Exception as e:
        print(s_project)
        continue

In [ ]:
df = pd.DataFrame.from_dict(median_results,orient='index')

In [ ]:
df.to_csv('result/cdom_score_100.csv',index=False)

In [ ]:
dom_score = []
goals = ['recall','precision','pf']
for row_id in range(df.shape[0]):
    project_name = df.index[row_id]
    row = df.iloc[row_id].tolist()
    wins = dominate(df,row,project_name,goals)
    dom_score.append(wins)
df['wins'] = dom_score

In [ ]:
df[df['wins'] == df.wins.max()]

In [ ]:
df_self = pd.read_pickle('data/self_100.pkl')
df_bellwether = pd.read_pickle('data/conv_bellwether_100.pkl')
metrices = df_self.keys()
median = 0
bell_results = {}
for s_project in projects:
    try:
        for metric in metrices:
            if metric not in bell_results.keys():
                bell_results[metric] = {}
            if s_project not in bell_results[metric]:
                bell_results[metric][s_project] = {}
            _self = df_self[metric][s_project]
            _bell = df_bellwether[metric]['disruptor'][s_project]
            bell_results[metric][s_project]['self'] = _self
            bell_results[metric][s_project]['bell'] = _bell
    except Exception as e:
        print(s_project)
        continue

In [ ]:
for metric in metrices:
    count = 0
    for key in bell_results[metric].keys():
        try:
            if  bell_results[metric][key]['self'] <= bell_results[metric][key]['bell']:
                count += 1
        except:
            continue
    print(metric, "wins", count)

In [ ]:
bell_results